In [2]:
import os
import keras
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers.merge import concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import  Input, Dense, Conv2D, Dropout, Flatten, MaxPooling2D, LSTM
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def make_model():
    k = float(np.random.rand()*1+0.2)
    print ('## k = %.3f' % k)
    winit1 = k/np.sqrt(5*5*1)
    winit2 = k/np.sqrt(5*5*64)
    winit3 = k/np.sqrt(5*5*128)
    
    cnn_input = Input(shape=(1,96,96))
    conv1 = Conv2D(filters=64, kernel_size=5, padding='same',
                    data_format='channels_first',
                    use_bias=False, 
                    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=winit1),
                    activation='relu')(cnn_input)
    maxpool1 = MaxPooling2D(pool_size=2, strides=(2,2))(conv1)
    
    conv2 = Conv2D(filters=128, kernel_size=5, padding='same',
                    data_format='channels_first',
                    use_bias=False,
                    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=winit2),
                    activation='relu')(maxpool1)
    maxpool2 = MaxPooling2D(pool_size=2, strides=(2,2))(conv2)
    
    conv3 = Conv2D(filters=256, kernel_size=5, padding='same',
                    data_format='channels_first',
                    use_bias=False,
                    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=winit3),
                    activation='relu')(maxpool2)
    maxpool3 = MaxPooling2D(pool_size=12, strides=(12,12))(conv3)
    flatten = Flatten()(maxpool3)

    winitD1 = k/np.sqrt(4032)
    winitD2 = k/np.sqrt(300)
    dropout = Dropout(0.5)(flatten)
    dense1 = Dense(300, activation='relu',
                    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05),
                    bias_initializer='Ones')(dropout)

    cnn_output = Dense(8, activation='softmax',
    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=winitD2),
    bias_initializer='Zeros')(dense1)

    cnn_model = Model(inputs=cnn_input, outputs=cnn_output)
    print(cnn_model.summary())
    
    # compile the model
    sgd = keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9)
    cnn_model.compile(loss='categorical_crossentropy', optimizer=sgd,
             metrics=['accuracy'])
    
    # LSTM Model
    rnn_input = Input(shape=(15, 65))
    lstm1 = LSTM(250, return_sequences=True)(rnn_input)
    feature_and_class = Input(shape=(15,4040,))
    merge = concatenate([lstm1, feature_and_class])
    lstm2 = LSTM(250, return_sequences=True)(merge)
    rnn_output = Dense(65, activation='softmax')(lstm2)
    
    rnn_model = Model(inputs=[rnn_input, feature_and_class], outputs=rnn_output)
    print(rnn_model.summary())
    rnn_model.compile(loss='categorical_crossentropy', optimizer=sgd,
             metrics=['accuracy'])
    
    return cnn_model, rnn_model

In [4]:
def load_data(dataset_path='./npy_files'):
    X = np.load(os.path.join(dataset_path,'X.npy'))
    y = np.load(os.path.join(dataset_path,'y.npy'))
    folds = np.load(os.path.join(dataset_path,'folds.npy'))
    feature_class = np.load(os.path.join(dataset_path, 'feature_class.npy'))
    facs = np.load(os.path.join(dataset_path,'facs_time.npy'))

    # rescale [0,255] -> [0,2]    
    X = X.astype('float32')/255*2

    # one-hot encode the labels
    num_classes = len(np.unique(y))
    y = keras.utils.to_categorical(y, num_classes)

    return X, y, folds, feature_class, facs

In [13]:
def make_facs_y(facs):
    facs_y = []
    for inst in facs:
        temp = list(inst)[1:]
        temp.append(list(np.zeros(65)))
        facs_y.append(temp)
    facs_y = np.array(facs_y)
    
    return facs_y

In [6]:
def make_cv_folds(X, y, feature_class, facs, folds, fold_num):
    X_train = []
    y_train = []
    feature_class_train = []
    facs_train = []
    
    X_valid = []
    y_valid = []
    feature_class_valid = []
    facs_valid = []
    
    for index, fold in enumerate(folds):
        if fold == fold_num:
            X_valid.append(X[index])
            y_valid.append(y[index])
            feature_class_valid.append(feature_class[index])
            facs_valid.append(facs[index])
        else:
            X_train.append(X[index])
            y_train.append(y[index])
            feature_class_train.append(feature_class[index])
            facs_train.append(facs[index])
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_valid = np.array(X_valid)
    y_valid = np.array(y_valid)
    
    feature_class_train = np.array(feature_class_train)
    feature_class_valid = np.array(feature_class_valid)

    facs_train = np.array(facs_train)
    facs_valid = np.array(facs_valid)
    facs_y_train = make_facs_y(facs_train)
    facs_y_valid = make_facs_y(facs_valid)

    return (X_train, y_train, 
            X_valid, y_valid,
            feature_class_train, feature_class_valid,
            facs_train, facs_valid,
            facs_y_train, facs_y_valid
           )

In [7]:
def load_cnn_model(model_path='./result/'):
    with open(os.path.join(model_path, 'model_architecture.json'), 'r') as f:
        model = model_from_json(f.read())

    model.load_weights(os.path.join(model_path, 'weights-42-0.90.hdf5'))
    
    return model

In [8]:
X, y, folds, feature_class, facs = load_data('./save_data/npy_files/')

_, rnn_model = make_model()
cnn_model = load_cnn_model('./result/test/')

## k = 0.859
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 96, 96)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 96, 96)        1600      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 48, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 48, 96)       102400    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 24, 96)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 24, 96)       409600    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 21, 2, 96)         0       

In [14]:
X_train, y_train, X_valid, y_valid, feature_class_train, feature_class_valid,facs_train, facs_valid,facs_y_train, facs_y_valid = make_cv_folds(X, y, feature_class,
                                            facs, folds, 0)

In [15]:
facs_train[1]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0

In [16]:
facs_y_train[1]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0

In [17]:
result_path_folder = './result/rnn_model'
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
filepath = os.path.join(result_path_folder,filepath)
checkpointer = ModelCheckpoint(filepath=filepath,
                        monitor='val_acc',
                        verbose=1, save_best_only=True)
 
hist = rnn_model.fit([facs_train,feature_class_train ], facs_y_train, batch_size=64, epochs=1000,
            validation_data=([facs_valid, feature_class_valid], facs_y_valid),
            callbacks=[checkpointer], verbose=2, shuffle=True)

Train on 1175 samples, validate on 132 samples
Epoch 1/1000
Epoch 00001: val_acc improved from -inf to 0.78182, saving model to ./result/rnn_model/weights-improvement-01-0.78.hdf5
 - 2s - loss: 1.5213 - acc: 0.7447 - val_loss: 1.1857 - val_acc: 0.7818
Epoch 2/1000
Epoch 00002: val_acc did not improve
 - 2s - loss: 1.0698 - acc: 0.7854 - val_loss: 1.0049 - val_acc: 0.7818
Epoch 3/1000
Epoch 00003: val_acc did not improve
 - 2s - loss: 0.9536 - acc: 0.7854 - val_loss: 0.9300 - val_acc: 0.7818
Epoch 4/1000
Epoch 00004: val_acc did not improve
 - 2s - loss: 0.8844 - acc: 0.7856 - val_loss: 0.8813 - val_acc: 0.7818
Epoch 5/1000
Epoch 00005: val_acc improved from 0.78182 to 0.79091, saving model to ./result/rnn_model/weights-improvement-05-0.79.hdf5
 - 2s - loss: 0.8296 - acc: 0.7884 - val_loss: 0.8366 - val_acc: 0.7909
Epoch 6/1000
Epoch 00006: val_acc improved from 0.79091 to 0.79596, saving model to ./result/rnn_model/weights-improvement-06-0.80.hdf5
 - 2s - loss: 0.7835 - acc: 0.7958 - v

Epoch 52/1000
Epoch 00052: val_acc did not improve
 - 2s - loss: 0.2504 - acc: 0.9379 - val_loss: 0.6780 - val_acc: 0.8500
Epoch 53/1000
Epoch 00053: val_acc improved from 0.85303 to 0.85354, saving model to ./result/rnn_model/weights-improvement-53-0.85.hdf5
 - 2s - loss: 0.2470 - acc: 0.9379 - val_loss: 0.7012 - val_acc: 0.8535
Epoch 54/1000
Epoch 00054: val_acc did not improve
 - 2s - loss: 0.2441 - acc: 0.9384 - val_loss: 0.6848 - val_acc: 0.8520
Epoch 55/1000
Epoch 00055: val_acc improved from 0.85354 to 0.85505, saving model to ./result/rnn_model/weights-improvement-55-0.86.hdf5
 - 2s - loss: 0.2397 - acc: 0.9406 - val_loss: 0.6794 - val_acc: 0.8551
Epoch 56/1000
Epoch 00056: val_acc did not improve
 - 2s - loss: 0.2371 - acc: 0.9425 - val_loss: 0.6841 - val_acc: 0.8510
Epoch 57/1000
Epoch 00057: val_acc did not improve
 - 2s - loss: 0.2353 - acc: 0.9409 - val_loss: 0.6765 - val_acc: 0.8540
Epoch 58/1000
Epoch 00058: val_acc did not improve
 - 2s - loss: 0.2296 - acc: 0.9442 - va

Epoch 114/1000
Epoch 00114: val_acc did not improve
 - 2s - loss: 0.1195 - acc: 0.9794 - val_loss: 0.7154 - val_acc: 0.8545
Epoch 115/1000
Epoch 00115: val_acc did not improve
 - 2s - loss: 0.1184 - acc: 0.9801 - val_loss: 0.7321 - val_acc: 0.8556
Epoch 116/1000
Epoch 00116: val_acc did not improve
 - 2s - loss: 0.1161 - acc: 0.9820 - val_loss: 0.7282 - val_acc: 0.8581
Epoch 117/1000
Epoch 00117: val_acc did not improve
 - 2s - loss: 0.1142 - acc: 0.9817 - val_loss: 0.7222 - val_acc: 0.8581
Epoch 118/1000
Epoch 00118: val_acc did not improve
 - 2s - loss: 0.1125 - acc: 0.9831 - val_loss: 0.7233 - val_acc: 0.8571
Epoch 119/1000
Epoch 00119: val_acc did not improve
 - 2s - loss: 0.1115 - acc: 0.9837 - val_loss: 0.7425 - val_acc: 0.8561
Epoch 120/1000
Epoch 00120: val_acc did not improve
 - 2s - loss: 0.1112 - acc: 0.9826 - val_loss: 0.7301 - val_acc: 0.8576
Epoch 121/1000
Epoch 00121: val_acc did not improve
 - 2s - loss: 0.1094 - acc: 0.9839 - val_loss: 0.7318 - val_acc: 0.8561
Epoch 12

KeyboardInterrupt: 

In [ ]:
rnn_model_json = rnn_model.to_json()
with open("./result/rnn_model/model_architecture.json", "w") as json_file:
    json_file.write(rnn_model_json)

In [ ]:
result_path = 'result/rnn_prototype'
hist = []
# CV Training
for times in range(10):
    result_path_times = os.path.join(result_path, str(times))
    hist_folds = []
    for val_fold in range(10):
        result_path_folds = os.path.join(result_path_times, str(val_fold))
        X_train, y_train, X_valid, y_valid = make_cv_folds(X, y, folds, val_fold)

        print('X_train shape :', X_train.shape)
        print('y_train shape :', y_train.shape)


        # printing number of training, validation, and test images
        print(X_train.shape[0], 'train samples')
        #print(X_test.shape[0], 'test samples')
        print(X_valid.shape[0], 'validation samples')
        #X_test = X_test.astype('float32')/255
        model = make_model()
        filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
        filepath = os.path.join(result_path_folds,filepath)
        checkpointer = ModelCheckpoint(filepath=filepath,
                                monitor='val_acc',
                                verbose=1, save_best_only=True)

        hist_folds.append( model.fit(X_train, y_train, batch_size=64, epochs=50,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpointer], verbose=2, shuffle=True) )
    hist.append(hist_folds)

In [ ]:
if __name__ == '__main__':
    # Data loading
    dataset_path = 'save_data/npy_files'
    result_path = 'result/'
    X = np.load(os.path.join(dataset_path,'X.npy'))
    y = np.load(os.path.join(dataset_path,'y.npy'))
    folds = np.load(os.path.join(dataset_path,'folds.npy'))
    
    # rescale [0,255] -> [0,2]    
    X = X.astype('float32')/255*2

    # one-hot encode the labels
    num_classes = len(np.unique(y))
    y = keras.utils.to_categorical(y, num_classes)
    
    

    hist = []
    # CV Training
    for times in range(10):
        result_path_times = os.path.join(result_path, str(times))
        hist_folds = []
        for val_fold in range(10):
            result_path_folds = os.path.join(result_path_times, str(val_fold))
            X_train, y_train, X_valid, y_valid = make_cv_folds(X, y, folds, val_fold)
            
            print('X_train shape :', X_train.shape)
            print('y_train shape :', y_train.shape)
            
            
            # printing number of training, validation, and test images
            print(X_train.shape[0], 'train samples')
            #print(X_test.shape[0], 'test samples')
            print(X_valid.shape[0], 'validation samples')
            #X_test = X_test.astype('float32')/255
            model = make_model()
            filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
            filepath = os.path.join(result_path_folds,filepath)
            checkpointer = ModelCheckpoint(filepath=filepath,
                                    monitor='val_acc',
                                    verbose=1, save_best_only=True)

            hist_folds.append( model.fit(X_train, y_train, batch_size=64, epochs=50,
                        validation_data=(X_valid, y_valid),
                        callbacks=[checkpointer], verbose=2, shuffle=True) )
        hist.append(hist_folds)
    
    
  
        